# Assignment 3.3

Group: Mathias Husted, Luciana Amarante, Leonie Kochs

We'll start by importing our functions from last time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def lagrange(x, nodes, k):
    f = 1 # Neutral element for multiplication
    for i in range(0, len(nodes)):
        if i != k:
            f *= (x-nodes[i]) / (nodes[k] - nodes[i])
    return f

def lagrange_interpolation(x, nodes, function_values):
    if len(nodes) != len(function_values):
        raise("Error: Nodes list must be as long as function_values")
    f = 0 # Neutral element for addition
    for i in range(0, len(nodes)):
        f += function_values[i] * lagrange(x, nodes, i)
    return f

For this assignment, we'll be numerically measuring the approximation error by using a discrete version of the Chebyshev distance (maximum metric). The interval given is $[a,b] = [-5,5]$

$$
||f||_h := \max\limits_{i \in \{0,...,1000\}} \Bigg | f \Bigg (a+\frac{i(b-a)}{1000} \Bigg ) \Bigg |
$$

Furthermore, we'll use the equidistant nodes

$$
x_i = a + \frac{i(b-a)}{n}, \text{ for } i \in {0,...,n}
$$

### a)
### i)
$$
f(x) := sin(x)
$$

We'll analyze the approximation error $||f-p_n||_h$ for $n \rightarrow \infty$ using $n \in \{1,...,100\}$

First, let's create our discrete Chebyshev distance function

In [2]:
# Interval
a,b = -5,5

def sin(x):
    return np.sin(x)

# Generates an interval corresponding to our customly defined Chebyshev distance metric
def generate_interval(a, b, n):
    interval = []
    for i in range(n):
        point = a + (i * (b - a) / n)
        interval.append(point)
    return np.array(interval)

# Returns ||f - p_n||_h
def chebyshev(a, b, func, x_nodes):
    
    y_nodes = func(x_nodes)
    # Generates the interval we want to approximate using our lagrange function
    x_interval = generate_interval(a, b, 1000)

    y_interval = func(x_interval)
    interpolated_values = lagrange_interpolation(x_interval, x_nodes, y_nodes)

    return np.max(np.abs(y_interval - interpolated_values))

Then, our n will run from 1 to 100 and approximate the error in each cycle using our custom Chebyshev distance metric.

In [3]:
def run_ns(func):
    for n in range(1,101):
        x_nodes = generate_interval(a, b, n)
        max = chebyshev(a, b, func, x_nodes)
        print(f"n = {n}: {max}")

run_ns(sin)

n = 1: 1.958923957594973
n = 2: 1.3196956618190432
n = 3: 7.869377063355043
n = 4: 0.8192497928342999
n = 5: 13.362592454457685
n = 6: 0.2864135975683023
n = 7: 6.511114764044004
n = 8: 0.053367677479241316
n = 9: 1.576712464501382
n = 10: 0.0063026124833258645
n = 11: 0.23157672493412929
n = 12: 0.0005191225552261258
n = 13: 0.02295806510789855
n = 14: 3.167506820345167e-05
n = 15: 0.001644099181382086
n = 16: 1.4928830240901192e-06
n = 17: 8.916259125857362e-05
n = 18: 5.607899100645852e-08
n = 19: 3.7920492667042893e-06
n = 20: 1.7321938328151987e-09
n = 21: 1.299329496218249e-07
n = 22: 2.214063377081743e-10
n = 23: 3.418290317469541e-09
n = 24: 1.2509050462128357e-09
n = 25: 3.380614788106584e-09
n = 26: 4.479442705651593e-09
n = 27: 1.1724250126299296e-08
n = 28: 2.5130542757878516e-08
n = 29: 5.1021512126325774e-08
n = 30: 1.5041575818219854e-07
n = 31: 1.0531807936864368e-07
n = 32: 3.6006396020304976e-07
n = 33: 4.142769823856085e-07
n = 34: 1.6319785735019465e-06
n = 35: 1.30

We can generally see that our function achieves its best performance between $n=20$ and $n=30$. After that, issues surrounding floating point numbers start to creep in and skew the result slightly, until it becomes completely unusable beyond $n\ge50$.

### ii)

Now we'll do the exact same thing, but with the function

$$
f(x)=(1+x^2)^{-1}
$$

In [4]:
def new_func(x):
    return 1/(1 + x*x)

run_ns(new_func)

n = 1: 0.9615384615384616
n = 2: 1.9210054950782784
n = 3: 0.7070135746606334
n = 4: 4.941647528794401
n = 5: 0.4326923076923077
n = 6: 12.807705900629244
n = 7: 0.24735860655931485
n = 8: 32.381736993290936
n = 9: 0.3002811297925705
n = 10: 79.87714063015612
n = 11: 0.556736492596072
n = 12: 193.24940435420837
n = 13: 1.0700405885115047
n = 14: 460.76178303139653
n = 15: 2.106800424232576
n = 16: 1086.53296545031
n = 17: 4.22345251877479
n = 18: 2540.4335856650937
n = 19: 8.575632648549394
n = 20: 5899.871352539262
n = 21: 17.60202211233917
n = 22: 13627.11165137655
n = 23: 36.39671823478378
n = 24: 31333.138611339902
n = 25: 75.76690554845013
n = 26: 71772.09622626341
n = 27: 158.69657366147194
n = 28: 163870.8746663565
n = 29: 333.9397252744126
n = 30: 373107.9029175543
n = 31: 704.0325556719419
n = 32: 847438.7614282141
n = 33: 1494.414704874117
n = 34: 1920656.82116091
n = 35: 3170.5436941165376
n = 36: 4344736.0597346015
n = 37: 6771.842982142151
n = 38: 9811531.109300327
n = 39:

With this more complex function, our algorithm starts to seriously deteriorate already at $n=4$, however up until about $n=10$, it seems to remain somewhat usable for uneven numbers. This suggests that the algorithm is unstable and could possibly be improved by rearranging multiplications/divisions. 